In [23]:
import pandas as pd
import glob
import csv

In [24]:
result = glob.glob( '**.csv' )
csv_df = {}

In [25]:
for fname in result:
    csv_df[fname.split('_')[-1].split('.')[0]] = pd.read_csv(fname,';')

In [26]:
def encode(mystring):
    mybytes = mystring.encode("utf-8")
    return int(int.from_bytes(mybytes, byteorder="little")%100000)

In [27]:
def decode(myint):
    recoveredbytes = myint.to_bytes((myint.bit_length() + 7) // 8, byteorder="little")
    return recoveredbytes.decode("utf-8")

In [28]:
def dict_encode(key, value):
    return value[key].map(lambda x:encode(x))

In [29]:
def min2h(key, value):
    return value[key].map(lambda x:round(x/60,3))

In [30]:
csv_df

{'nodes':             ID  type      Pd   Qd   Gs   Bs  area_num   Vm   Va  baseKV  zone  \
 0   trans_grid     3     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 1       T1-K1a     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 2       K1a-E1     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 3       E1-K1b     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 4       K1b-K1     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 5       K1-K1c     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 6          NL1     1  5000.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 7       K2a-K2     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 8          NL2     1  4000.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 9       K3a-K3     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 10      K3-K3b     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 11    

In [31]:
# I convert repair_time and failure_time in reldata from minutes to hours
# I have to check if frequencies have to be multiplied for the length

for (key, value) in csv_df.items():
    print(key)
    if key == 'nodes':
        value['ID'] = dict_encode('ID', value)
    elif key=='lines' or key== 'transformers' or key == 'switchgear':
        value['f_bus'] = dict_encode('f_bus', value)
        value['t_bus'] = dict_encode('t_bus', value)
    elif key == 'loads' or key=='gen':
        value['bus'] = dict_encode('bus', value)
    elif key=='reldata':
        value['f_bus'] = dict_encode('f_bus', value)
        value['t_bus'] = dict_encode('t_bus', value)
        value['repairTime'] = min2h('repairTime', value)
        value['sectioningTime'] = min2h('sectioningTime', value)
        
        
        
    

nodes
switchgear
gen
lines
loads
reldata
transformers


In [32]:
csv_df

{'nodes':        ID  type      Pd   Qd   Gs   Bs  area_num   Vm   Va  baseKV  zone  \
 0   29364     3     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 1   44724     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 2   48779     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 3   72485     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 4   18091     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 5     267     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 6   30798     1  5000.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 7   80587     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 8   96334     1  4000.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 9    8619     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 10   7595     1     0.0  0.0  0.0  0.0         0  1.0  0.0    22.0     0   
 11  61870     1  3000.0  0.0  0.0  0.0         0  1.0  0.0    22.0

In [33]:
for (key, value) in csv_df.items():
    value.to_csv(key+'.csv',';', index=False)